In [43]:
import os
import pandas as pd
import optuna
import numpy as np
import openml
import logging
import matplotlib.pyplot as plt
import seaborn as sns
import kaleido
import plotly
import scipy.stats as stats
from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from optuna.samplers import TPESampler, CmaEsSampler
from xgboost import XGBClassifier
from plotly.io import show
from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from sklearn.metrics import roc_auc_score

In [44]:
dataset_ids = [
    1590,   # Adult
    1510,   # Breast Cancer Wisconsin (Diagnostic)
    1461,   # Bank Marketing
    24,     # Mushroom
    40945   # Titanic
]

In [45]:
def fetch_and_prepare(openml_id):
    global y
    global X
    dataset = openml.datasets.get_dataset(openml_id)
    print(f">>> {dataset.name} (ID: {openml_id})")
    X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute, dataset_format='dataframe')
    for col in X.select_dtypes(include=['category', 'object']):
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
    imputer = SimpleImputer(strategy='mean')
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
    #scaler = MinMaxScaler()
    #X_imputed = pd.DataFrame(scaler.fit_transform(X_imputed), columns=X.columns)
    if y.dtype == 'category':
        y = LabelEncoder().fit_transform(y)
    zbior = X_imputed['Target'] = y
    return X_imputed

In [46]:
zbior = fetch_and_prepare(40945)

>>> Titanic (ID: 40945)


In [47]:
X_train, X_test, y_train, y_test = train_test_split(zbior.iloc[:, :-1], zbior['Target'], test_size=0.30, random_state=42)

def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 921, 4551) 
    eta = trial.suggest_float("eta", 0.002, 0.355, log=True)             
    subsample = trial.suggest_float("subsample", 0.545, 0.958)  
    max_depth = trial.suggest_int("max_depth", 6, 14) 
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.419, 0.864) 

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    wyniki = []

    for train_index, test_index in kf.split(X_train):
        X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
        y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

        X_train_fold = X_train_fold.reset_index(drop=True)
        X_test_fold = X_test_fold.reset_index(drop=True)
        y_train_fold = y_train_fold.reset_index(drop=True)
        y_test_fold = y_test_fold.reset_index(drop=True)

        X_train_fold = list(zip(*[X_train_fold[col] for col in X_train_fold]))
        X_test_fold = list(zip(*[X_test_fold[col] for col in X_test_fold]))

        XGBoost = XGBClassifier(n_estimators=n_estimators,
        eta=eta, subsample=subsample, max_depth=max_depth, colsample_bytree=colsample_bytree, random_state=42, n_jobs=6, eval_metric="auc")


        model = XGBoost.fit(X_train_fold, y_train_fold)


        y_pred_proba = model.predict_proba(X_test_fold)[:, 1]
        auc = roc_auc_score(y_test_fold, y_pred_proba)
                                            
        wyniki.append(auc)

    rezultat = np.mean(wyniki)
    return rezultat

In [48]:
study = optuna.create_study(direction='maximize',sampler=TPESampler())
wynik = study.optimize(objective, n_trials=32)

[I 2025-11-14 23:12:56,418] A new study created in memory with name: no-name-b538ea7e-5fbc-4ecc-aa46-0271583d8952
[I 2025-11-14 23:13:01,012] Trial 0 finished with value: 0.9920894560731695 and parameters: {'n_estimators': 3544, 'eta': 0.0027464029890972895, 'subsample': 0.5908905148277918, 'max_depth': 12, 'colsample_bytree': 0.5096697434822134}. Best is trial 0 with value: 0.9920894560731695.
[I 2025-11-14 23:13:01,995] Trial 1 finished with value: 0.9866594424886802 and parameters: {'n_estimators': 1445, 'eta': 0.2829554670578166, 'subsample': 0.6374859567094283, 'max_depth': 6, 'colsample_bytree': 0.678109279987587}. Best is trial 0 with value: 0.9920894560731695.
[I 2025-11-14 23:13:03,527] Trial 2 finished with value: 0.987540875722668 and parameters: {'n_estimators': 1531, 'eta': 0.05328378144047691, 'subsample': 0.7282709155475093, 'max_depth': 10, 'colsample_bytree': 0.6493819805487655}. Best is trial 0 with value: 0.9920894560731695.
[I 2025-11-14 23:13:07,023] Trial 3 finish

In [49]:
study.best_params

{'n_estimators': 2264,
 'eta': 0.002168571549951855,
 'subsample': 0.7028115224702192,
 'max_depth': 8,
 'colsample_bytree': 0.5565780545300623}

In [50]:
#Defaultowe
XGBoost = XGBClassifier(n_estimators= 500,
        eta= 0.3, subsample=1, max_depth= 6, colsample_bytree=1, random_state=42, n_jobs=6, eval_metric="auc")
model = XGBoost.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(auc)

0.9922865595942518


In [52]:
#Optymalne
XGBoost = XGBClassifier(n_estimators=2264,
        eta=0.002168571549951855, subsample=0.7028115224702192, max_depth=8, colsample_bytree=0.7028115224702192, random_state=42, n_jobs=6, eval_metric="auc")
model = XGBoost.fit(X_train, y_train)
y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(auc)

0.9945583262890956
